## 결정 트리, 랜덤 포레스트, 에이다부스트. 
#### 필요한 패키지를 불러온다:

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics, preprocessing
from scipy.stats import itemfreq

#### 데이터를 불러온다:

In [ ]:
os.chdir(r"D:\python_ML\python_ml_basic\data")

In [ ]:
# df = pd.read_csv('data_spam.csv', header='infer',encoding='ISO-8859-1')
df = pd.read_csv('data_spam.csv', header='infer',encoding='latin1')

In [ ]:
X=np.array(df.drop(columns='is_spam'))
Y=np.array(df.is_spam)
header = df.columns
headerX = df.drop(columns='is_spam').columns

'yes', 'no' 레이블을 숫자로 변환:

In [ ]:
LE = preprocessing.LabelEncoder()
Y = LE.fit_transform(Y)

In [ ]:
table = itemfreq(Y)
table

NaN이 있으면 채워 넣음:

In [ ]:
IPT = preprocessing.Imputer()
X = IPT.fit_transform(X)

#### 데이터 전처리:

In [ ]:
X = preprocessing.scale(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=3)

#### Tree 적용 (교차검증 최적화 포함):

In [ ]:
depth_grid = np.arange(2,31,2)
parameters = {'max_depth':depth_grid}

In [ ]:
gridCV = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10)
gridCV.fit(X_train, Y_train);
best_depth = gridCV.best_params_['max_depth']

In [ ]:
print("Tree best depth : " + str(best_depth))

In [ ]:
DTC_best = DecisionTreeClassifier(max_depth=best_depth)
DTC_best.fit(X_train, Y_train);
Y_pred = DTC_best.predict(X_test)
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

#### Random Forest 적용 (교차검증 최적화 포함):

Tree 하고는 스텝에서 조금 차이가 있음:

In [ ]:
estimator_grid = np.arange(1, 30, 5)
depth_grid = np.arange(1, 10, 2)
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_depth = gridCV.best_params_['max_depth']

In [ ]:
print("Random Forest best n estimator : " + str(best_n_estim))
print("Random Forest best depth : " + str(best_depth))

In [ ]:
RF_best = RandomForestClassifier(max_depth=best_depth,n_estimators=best_n_estim,random_state=3)
RF_best.fit(X_train, Y_train);
Y_pred = RF_best.predict(X_test)
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

#### AdaBoost 적용 (교차검증 최적화 포함): <br>
디폴트: base_estimator=DecisionTreeClassifier 

In [ ]:
estimator_grid = np.arange(30, 80, 10)
learning_rate_grid = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
parameters = {'n_estimators': estimator_grid, 'learning_rate': learning_rate_grid}
gridCV = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_learn_rate = gridCV.best_params_['learning_rate']

In [ ]:
print("Ada Boost best n estimator : " + str(best_n_estim))
print("Ada Boost best learning rate : " + str(best_learn_rate))

In [ ]:
AB_best = AdaBoostClassifier(n_estimators=best_n_estim,learning_rate=best_learn_rate,random_state=3)
AB_best.fit(X_train, Y_train);
Y_pred = AB_best.predict(X_test)
print( "Ada Boost best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))